In [1]:
# Import necessary libraries
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.preprocessing import StandardScaler





C:\Users\Faizan Ahmad\AppData\Local\Temp\ipykernel_13824\1620173721.py:2: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [3]:
# Load the datasets
customers = pd.read_csv('Customers.csv')
products = pd.read_csv('Products.csv')
transactions = pd.read_csv('Transactions.csv')



In [4]:
# Clean column names (fix malformed ones if needed)
customers.columns = ['CustomerID', 'CustomerName', 'Region', 'SignupDate']
products.columns = ['ProductID', 'ProductName', 'Category', 'Price']
transactions.columns = ['TransactionID', 'CustomerID', 'ProductID', 'TransactionDate', 'Quantity', 'TotalValue', 'Price']



In [5]:
# Merge datasets
data = transactions.merge(customers, on="CustomerID").merge(products, on="ProductID")



In [6]:
# Create aggregated features for each customer
customer_features = data.groupby('CustomerID').agg({
    'TotalValue': 'sum',
    'TransactionID': 'count',
    'Category': pd.Series.nunique,
    'ProductID': pd.Series.nunique,
    'Quantity': 'sum'
}).reset_index()



In [7]:
# Rename columns for clarity
customer_features.columns = [
    'CustomerID', 'TotalSpending', 'TotalTransactions', 
    'UniqueCategoriesPurchased', 'UniqueProductsPurchased', 'TotalQuantityPurchased'
]




In [8]:
# Normalize the data
scaler = StandardScaler()
scaled_features = scaler.fit_transform(customer_features.iloc[:, 1:])




In [9]:
# Calculate cosine similarity
similarity = cosine_similarity(scaled_features)



In [10]:
# Function to find top N similar customers
def find_similar_customers(customer_id, top_n=3):
    idx = customer_features[customer_features['CustomerID'] == customer_id].index[0]
    scores = list(enumerate(similarity[idx]))
    scores = sorted(scores, key=lambda x: x[1], reverse=True)[1:top_n+1]  # Exclude self
    return [(customer_features.iloc[i[0]]['CustomerID'], round(i[1], 4)) for i in scores]



In [12]:
# Generate lookalike results for the first 20 customers
lookalikes = {customer_id: find_similar_customers(customer_id) 
              for customer_id in customer_features['CustomerID'][:20]}

In [13]:
print(lookalikes);


{'C0001': [('C0056', 0.9381), ('C0164', 0.9299), ('C0189', 0.9024)], 'C0002': [('C0199', 0.9932), ('C0142', 0.9886), ('C0010', 0.9741)], 'C0003': [('C0027', 0.9139), ('C0166', 0.8256), ('C0129', 0.7775)], 'C0004': [('C0124', 0.9907), ('C0195', 0.9886), ('C0156', 0.9882)], 'C0005': [('C0131', 0.9997), ('C0058', 0.9994), ('C0150', 0.9987)], 'C0006': [('C0079', 0.9999), ('C0196', 0.9508), ('C0026', 0.8809)], 'C0007': [('C0080', 0.9959), ('C0078', 0.9957), ('C0020', 0.9949)], 'C0008': [('C0162', 0.9629), ('C0093', 0.9611), ('C0017', 0.9568)], 'C0009': [('C0083', 0.9967), ('C0198', 0.9812), ('C0071', 0.977)], 'C0010': [('C0142', 0.9774), ('C0002', 0.9741), ('C0199', 0.9413)], 'C0011': [('C0170', 0.9687), ('C0200', 0.9476), ('C0114', 0.9433)], 'C0012': [('C0101', 0.9847), ('C0087', 0.9767), ('C0156', 0.9725)], 'C0013': [('C0099', 0.9963), ('C0105', 0.9953), ('C0155', 0.9913)], 'C0014': [('C0033', 0.9998), ('C0060', 0.9996), ('C0150', 0.9995)], 'C0015': [('C0123', 0.9991), ('C0095', 0.9988), 

AttributeError: 'dict' object has no attribute 'describe'